## Spline-background Demo
<div class="alert alert-block alert-info">

<p><b>Information:</b> This notebook shows behaviour of the background spline method implemented in <code>scipp</code>.<br> 
A random, noisy dataset is generated from a gaussian distribution. A second non-random dataset is generated as well.<br>
Splines for both cases are generated with <code>scipp</code> and <code>Mantid</code> and compared and plotted.</p>

<p><b>Requirements:</b> To run this notebook, you need <code>mantid</code>, <code>scipp</code>, <code>matplotlib</code> and <code>numpy</code> installed as well as the Python script <code>spline_background.py.</code> placed in the same folder as this notebook.</p>
</div>

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
%matplotlib notebook

import numpy as np
import mantid.simpleapi as mapi
import scipp as sc

import spline_background

In [ ]:
# Create a workspace with Gaussian peak
x = np.arange(30)
y0 = 20. + 50. * np.exp(-(x - 8.)**2. / 120.)
err0 = np.sqrt(y0)

y = y0 + err0 * np.random.normal(size=len(err0))
err = np.sqrt(y)

# Create Mantid workspace
w_1 = mapi.CreateWorkspace(DataX=x, 
                           DataY=y, 
                           DataE=err, 
                           NSpec=1, 
                           UnitX='tof')

In [ ]:
# Create a damped sinusoidal example
N = 100
x_2 = np.arange(N)
y_20 = np.zeros(N)
y_2 = y_20

for i, item in enumerate(y_2):
    xi = (15. / (N-1)) * i
    y_2[i] = np.cos(xi) * np.exp(-0.1 * xi)
    
err_2 = np.sqrt(y_2 * y_2)

# Create Mantid workspace
w_2 = mapi.CreateWorkspace(DataX=x_2, 
                           DataY=y_2, 
                           DataE=err_2, 
                           NSpec=1, 
                           UnitX='tof')    

In [ ]:
def plot_mantid_spline(w, x, y):
    """ 
    Plot of original data with input and output of Mantid's SplineBackground 
    
    w: Mantid workspace to be plotted
    
    x: x-values of the original data (without noise)
    
    y: y-values of the original data (without noise)
    """
    fig, ax = plt.subplots()
    ax.plot(x, y, 'k', label='Original data')
    ax.plot(w.readX(0), w.readY(0), 'x', label='Original noisy data') # plot the initial distribution with black line

    spline = mapi.SplineBackground(w)
    ax.plot(spline.readX(0), spline.readY(0), label='Splined data')
    
    ax.grid()
    ax.legend()
    ax.set_title('Mantid')
    ax.set_xlabel('Time-of-flight ($\mu$s)')

### Reference Mantid plot, SplineBackground method

In [ ]:
plot_mantid_spline(w_1, x, y0)

### Comparison between Mantid SplineBackground and scipp spline_background methods

In [ ]:
def plot_comparison(x, y, err):
    """
    Compare output of spline from Mantid and scipp with input
     
    x: x-values of input data 
    
    y: y-values of input data
    
    err: err-values of input datta

    """
    fig, ax = plt.subplots()
    
    # plot the initial distribution with black line
    ax.plot(x, y, 'kx', label='input') 
    
    # scipp: calculate input dataset, calculate spline and plot output
    da = sc.DataArray(data=sc.Variable(dims=['x'], 
                                       values=y,
                                       variances=err**2), 
                      coords={'x': sc.Variable(dims=['x'],
                                               values=x)}
                     )

    output_array = spline_background.bspline_background(da, 'x', smoothing_factor=1)
    x_sp = output_array.coords['x'].values
    y_sp = output_array.values    
    
    ax.plot(x_sp, y_sp, 'r-', label='scipp spline') 
    
    # mantid: calculate input dataset, calculate spline and plot output
  
    w= mapi.CreateWorkspace(DataX=x, 
                           DataY=y, 
                           DataE=err, 
                           NSpec=1, 
                           UnitX='tof')
    
    output_mantid = mapi.SplineBackground(w, NCoeff=20)
    ax.plot(output_mantid.readX(0), output_mantid.readY(0), 'b-', label='mantid spline') # plot the mantid spline
    
    ax.grid()
    ax.legend()
    ax.set_title('Scipp and Mantid')
    ax.set_xlabel('Time-of-flight ($\mu$s)')

In [ ]:
plot_comparison(x, y, err)

In [ ]:
plot_comparison(x_2, y_2, err_2)